<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Exercices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Installation des bibliothèques nécessaires
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# 2. Chargement du dataset
from datasets import load_dataset
from langchain.schema import Document

print("Chargement du dataset Dolly...")
dataset = load_dataset(
    "databricks/databricks-dolly-15k",
    split="train",
    streaming=True
)

# Conversion en Documents Langchain
data = [
    Document(
        page_content=item["context"],
        metadata={"source": "dolly"}
    )
    for item in list(dataset.take(1000))  # On prend les 1000 premiers exemples
]

print("\nVérification des premiers documents :")
print(data[:2])

Chargement du dataset Dolly...

Vérification des premiers documents :
[Document(metadata={'source': 'dolly'}, page_content="Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."), Document(metadata={'source': 'dolly'}, page_content='')]


In [18]:
# 3. Découpage des documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)
docs = text_splitter.split_documents(data)
print("\nPremier document découpé :")
print(docs[0])


Premier document découpé :
page_content='Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.' metadata={'source': 'dolly'}


In [5]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [19]:
# 4. Création des embeddings
from langchain.embeddings import HuggingFaceEmbeddings

modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Test des embeddings
text = "This is a test document."
query_result = embeddings.embed_query(text)
print("\nTest d'embedding (3 premières valeurs) :")
print(query_result[:3])



Test d'embedding (3 premières valeurs) :
[-0.038338541984558105, 0.12346471846103668, -0.02864297851920128]


In [20]:
# 5. Création du vector store
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)
print("\nVector store créé avec succès")



Vector store créé avec succès


In [9]:
!pip install -q langchain-huggingface


In [40]:
# 6. Préparation du modèle LLM
# Import des classes nécessaires
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from langchain_huggingface import HuggingFacePipeline  # Note: using langchain_huggingface instead of deprecated langchain

# Chargement du tokenizer et du modèle question-answering
tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")
model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

# Création du pipeline question-answering
model_name = "Intel/dynamic_tinybert"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)
Youtubeer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Création du wrapper Langchain pipeline
llm = HuggingFacePipeline(
    pipeline=Youtubeer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)


Invalid model-index. Not loading eval results into CardData.
Invalid model-index. Not loading eval results into CardData.
Device set to use cpu


In [48]:
# 7. Construction de la chaîne de Retrieval QA avec une approche plus directe
def get_answer_with_youtubeer(question):
    # Utilisation du retriever existant
    docs = retriever.get_relevant_documents(question)

    # Préparation du contexte
    context = docs[0].page_content if docs else ""

    # Utilisation directe du pipeline Youtubeer configuré au point 6
    # avec le format correct question/contexte
    response = Youtubeer(
        {
            "question": question,
            "context": context
        }
    )

    return response

In [49]:
# 8. Test du système
question = "What is cheesemaking?"
try:
    resultat = get_answer_with_youtubeer(question)
    print("\nQuestion :", question)
    print("Réponse :", resultat['answer'])
    if resultat['score'] < 0.5:  # Si le score est faible
        print("\nNote : La réponse a un faible niveau de confiance. Dans le contexte de votre analyse des plats traditionnels, il serait pertinent d'ajouter des documents sur les processus de fabrication des fromages.")
except Exception as e:
    print(f"Erreur : {str(e)}")



/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(



Question : What is cheesemaking?
Réponse : A grilled cheese sandwich is made by placing a cheese filling

Note : La réponse a un faible niveau de confiance. Dans le contexte de votre analyse des plats traditionnels, il serait pertinent d'ajouter des documents sur les processus de fabrication des fromages.
